In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = '../data/Blight/Corn_Blight (223).JPG'
cv2image = cv2.imread(image)

def show_image(image):
    if image.ndim > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(image)

show_image(cv2image)

In [ ]:
hsv = cv2.cvtColor(cv2image, cv2.COLOR_BGR2HSV)
mask_green = cv2.inRange(hsv, (36,0,0), (86,255,255)) # green
mask_brown = cv2.inRange(hsv, (8, 60, 20), (30, 255, 200)) # brown
mask_yellow = cv2.inRange(hsv, (14, 20, 55), (40, 255, 255)) # yellow

mask = cv2.bitwise_or(mask_green, mask_brown)
mask = cv2.bitwise_or(mask, mask_yellow)

show_image(mask)

In [ ]:
mask_copy = mask.copy()

mask_copy = cv2.copyMakeBorder(mask_copy, 20, 20, 20, 20, cv2.BORDER_CONSTANT, value=0)
contours, _ = cv2.findContours(mask_copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(mask_copy, contours, -1, (0, 255, 0), 2)
show_image(mask_copy)


In [ ]:
# add 20 pixels of padding to the mask
mask = cv2.copyMakeBorder(mask, 20, 20, 20, 20, cv2.BORDER_CONSTANT, value=0)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = [c - [20, 20] for c in contours]
mask = mask[20:-20, 20:-20]

copy = cv2image.copy()
contours = [c for c in contours if cv2.contourArea(c) > 100]
cv2.drawContours(copy, contours, -1, (0, 255, 0), 2)
show_image(copy)

# apply contour mask
mask = np.zeros_like(mask)
cv2.drawContours(mask, contours, -1, 255, -1)
show_image(mask)

In [ ]:
result = cv2.bitwise_and(cv2image, cv2image, mask=mask)
show_image(result)

In [ ]:
masked_image_path = "../segdata/" + "/".join(image.split("/")[2:])
cv2.imwrite(masked_image_path, result)

In [ ]:
# for i in range(49, 561):
for i in range(1, 600):
# for i in range(224, 1095):
    image = f'../diffBg/Gray_Leaf_Spot/Corn_Gray_Spot ({i}).JPG'
    # image = f'../diffBg/Common_Rust/Corn_Common_Rust ({i}).JPG'
    # image = f'../diffBg/Blight/Corn_Blight ({i}).JPG'
    # image = f'../diffBg/Healthy/healthy{i}.png'
    print(image)
    cv2image = cv2.imread(image)
    if cv2image is None:
        image = f'../diffBg/Gray_Leaf_Spot/Corn_Gray_Spot ({i}).jpg'
        # image = f'../diffBg/Common_Rust/Corn_Common_Rust ({i}).jpg' 
        # image = f'../diffBg/Blight/Corn_Blight ({i}).jpg'
        # image = f'../diffBg/Healthy/healthy{i}.jpg'
        cv2image = cv2.imread(image) 
        if cv2image is None:
            continue
    hsv = cv2.cvtColor(cv2image, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(hsv, (36,0,0), (86,255,255)) # green
    mask_brown = cv2.inRange(hsv, (8, 60, 20), (30, 255, 200)) # brown
    mask_yellow = cv2.inRange(hsv, (14, 20, 64), (40, 255, 255)) # yellow

    mask = cv2.bitwise_or(mask_green, mask_brown)
    mask = cv2.bitwise_or(mask, mask_yellow)

    mask = cv2.copyMakeBorder(mask, 20, 20, 20, 20, cv2.BORDER_CONSTANT, value=0)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = [c - [20, 20] for c in contours]
    mask = mask[20:-20, 20:-20]

    copy = cv2image.copy()
    contours = [c for c in contours if cv2.contourArea(c) > 100]
    cv2.drawContours(copy, contours, -1, (0, 255, 0), 2)
    mask = np.zeros_like(mask)
    cv2.drawContours(mask, contours, -1, 255, -1)


    # for y in range(cv2image.shape[0]):
    #     start = None
    #     end = None
    #     for x in range(cv2image.shape[1]):
    #         if mask[y, x] > 0:
    #             if start is None:
    #                 start = x
    #             else:
    #                 end = x
    #     if start is None or end is None:
    #         continue
    #     for x in range(cv2image.shape[1]):
    #         if x > start and x < end:
    #             mask[y, x] = 255
    res = cv2.bitwise_and(cv2image,cv2image, mask=mask)
    masked_image_path = "../segDiffBg/" + "/".join(image.split("/")[2:])

    cv2.imwrite(masked_image_path, res)
    print(f"Saved {masked_image_path}")